In [1]:
# First, import necessary packages:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from iso3166 import countries # install iso3166：pip install iso3166
import re # regular expression operation
from sqlalchemy import create_engine# connect to sql database
import datetime # time convert
from dateutil.parser import parse # time convert

# Data Cleaning
This part is to clean the glassdoor.csv by:
1. Standardize the map.country column;
2. Add a job type column to standarize the jobTitle column;
3. Identify the skills from jobDescription context column.

In [2]:
# Loading the main file for glassdoor listings
glassdoor_full = pd.read_csv('glassdoor.csv')
#files needed later:'glassdoor_benefits_comments.csv','glassdoor_reviews.csv','glassdoor_overview_competitors.csv'

## 1. Country

In [3]:
# Selecting country column
country_columns =['map.country']
glassdoor_country = glassdoor_full[country_columns].copy()
glassdoor_country.head() # inspect the column

,map.country
0,Poland
1,NaN
2,Italy
3,PK
4,IN


In [4]:
#drop null values.
#glassdoor_country['map.country'].isnull().sum() #50028 null values
countriesData = glassdoor_country.dropna() 
countriesData #115262rows

,map.country
0,Poland
2,Italy
3,PK
4,IN
6,IL
...,...
165284,Hungary
165286,JP
165287,BE
165288,NL


In [5]:
#Noticing there is not a proper convention for the countries names:
countriesData['map.country'].unique()

array(['Poland', 'Italy', 'PK', 'IN', 'IL', 'FR', 'ID', 'HK', 'US', 'AE',
       'BE', 'South Korea', 'France', 'United Kingdom', 'Malaysia', 'ZM',
       'NG', 'SG', 'ES', 'CA', 'Austria', 'Ukraine', 'AT', 'RO', 'DE',
       'SA', 'IT', 'Germany', 'MY', 'AR', 'ZA', 'UA', 'PL', 'Israel',
       'Thailand', 'TH', 'New Zealand', 'Switzerland', 'CHE',
       'Netherlands', 'Hong Kong', 'LU', 'Belgium', 'Nigeria',
       'Indonesia', 'UK', 'Saudi Arabia', 'GB', 'IE', 'GH', 'UG', 'EG',
       'Australia', 'Nederland', 'AU', 'Mexico', 'India', 'TW', 'Sweden',
       'PH', 'SK', 'CN', 'Lithuania', 'MM', 'NL', 'Saint Kitts And Nevis',
       'CZ', 'Turkey', 'ro', 'LT', 'UZ', 'Finland', 'JP', 'United States',
       'Russia', 'USA', 'RU', 'NZ', 'Portugal', 'Kingdom', 'Spain', 'CH',
       'DK', 'TR', 'South Africa', 'Serbia', 'Zimbabwe', 'Kenya', 'Japan',
       'Slovakia', 'Singapore', 'BR', 'China', 'QA', 'Hungary', 'BY',
       'Romania', 'nl', 'Taiwan', 'Greece', 'MX', 'Ireland',
       'UN

In [6]:
#Use iso3166 get function to convert country name
def rename_code(country):
    try:
        return countries.get(country).alpha3 # Adjust the alpha to get the format needed
    except:
        return (np.nan)
    
def rename_name(country):
    try:
        return countries.get(country).name # Adjust the alpha to get the format needed
    except:
        return (np.nan)

In [7]:
countriesData['country_code'] = countriesData['map.country'].apply(rename_code)
countriesData['country_name'] = countriesData['map.country'].apply(rename_name)
countriesData = countriesData.dropna()
countriesData#110172

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,map.country,country_code,country_name
0,Poland,POL,Poland
2,Italy,ITA,Italy
3,PK,PAK,Pakistan
4,IN,IND,India
6,IL,ISR,Israel
...,...,...,...
165284,Hungary,HUN,Hungary
165286,JP,JPN,Japan
165287,BE,BEL,Belgium
165288,NL,NLD,Netherlands


In [8]:
#add new columns in main table
glassdoor_full['country_code'] = countriesData['country_code']
glassdoor_full['country_name'] = countriesData['country_name']

## 2. Job title

In [9]:
# Selecting only the columns that are related to jobs table
job_columns =['gaTrackerData.jobId.long','header.jobTitle','job.description']
glassdoor_job = glassdoor_full[job_columns].copy()

# Rename columns for more meaningful names
glassdoor_job.columns = ['JobId','JobTitle', 'JobDescription']
glassdoor_job.head()

,JobId,JobTitle,JobDescription
0,3.227100e+09,Biogas Project Development Manager,We are the drivers of the future who offer a g...
1,3.406582e+09,Quality Manager - Boehringer Ingelheim Healthc...,<strong></strong>\n<br/><br/>\n<strong></stron...
2,3.230738e+09,"Senior Software Engineer (PHP, Elixir, Python)",Dottori.it &egrave; il motore di ricerca che t...
3,3.406677e+09,Senior SQA Engineer,Codup is looking for a Senior SQA Engineer for...
4,3.201516e+09,Research Manager,<strong>Department:</strong> Research &amp; Op...


In [10]:
# List of job types that we need
job_titles = ['data scientist', 'software engineer', 'data analyst', 'research scientist', 'business analyst',
              'data engineer', 'statistician', 'dba', 'database engineer', 'machine learning engineer']
# Creating masks for each job type to identify where they appear
job_masks = [glassdoor_job.JobTitle.str.contains(job_title,flags=re.IGNORECASE,regex=True) for job_title in job_titles]
# Combining all masks where any value is True, return True
combined_mask = np.vstack(job_masks).any(axis=0)
glassdoor_job[combined_mask]

,JobId,JobTitle,JobDescription
2,3.230738e+09,"Senior Software Engineer (PHP, Elixir, Python)",Dottori.it &egrave; il motore di ricerca che t...
13,3.370129e+09,Big Data Analyst,Amaris est une soci&eacute;t&eacute; ind&eacut...
15,3.041354e+09,Business Analyst IFRS 17 Implementation Paris F/H,<div>Business Analyst Paris F/H</div>\n<br/>\n...
16,3.403542e+09,Data Scientist media,<ul>\n<li>*Data Scientist media**</li>\n<li>*L...
19,3.407938e+09,Software Engineer,<h1>Software Engineer</h1>\n\n<ul>\n<li>Singap...
...,...,...,...
165271,3.187583e+09,Senior Software Engineer,<strong>Senior Software Engineer &ndash; Dubli...
165272,3.410744e+09,Data Analyst,Job Description<br/><br/>Ubisoft is committed ...
165275,3.290131e+09,Senior Manager-Data Engineering,<div>\n<div><div><div>Software Development Eng...
165279,3.303312e+09,IT Business Analyst,<strong>Job DescriptionDuties/Responsibilities...


In [11]:
# Now lets clean job titles even further and remove any word that doesn't match the terms from Kaggle survey question 5
job_titles_regex = '|'.join(job_titles)
glassdoor_job.JobTitle = glassdoor_job.JobTitle.str.findall(job_titles_regex, flags=re.IGNORECASE)
glassdoor_job.JobTitle = glassdoor_job.JobTitle.str[0]
glassdoor_job.JobTitle = glassdoor_job.JobTitle.str.title()
glassdoor_job

,JobId,JobTitle,JobDescription
0,3.227100e+09,NaN,We are the drivers of the future who offer a g...
1,3.406582e+09,NaN,<strong></strong>\n<br/><br/>\n<strong></stron...
2,3.230738e+09,Software Engineer,Dottori.it &egrave; il motore di ricerca che t...
3,3.406677e+09,NaN,Codup is looking for a Senior SQA Engineer for...
4,3.201516e+09,NaN,<strong>Department:</strong> Research &amp; Op...
...,...,...,...
165285,2.939045e+09,NaN,<strong>Responsibilities</strong>\n\n<ul>\n<li...
165286,3.301658e+09,NaN,<strong>Job Description</strong><br/><br/>Make...
165287,3.386398e+09,NaN,Functie\n<br/>\nJe ondersteunt de productiever...
165288,3.402587e+09,NaN,Het Instituut voor Engineering is &eacute;&eac...


In [12]:
# group DBA + Data Engineer
glassdoor_job.JobTitle = glassdoor_job.JobTitle.replace('Dba', 'Data Engineer/DBA')
glassdoor_job.JobTitle = glassdoor_job.JobTitle.replace('Database Engineer', 'Data Engineer/DBA')
glassdoor_job.JobTitle = glassdoor_job.JobTitle.replace('Data Engineer', 'Data Engineer/DBA')

# And group Statistician + Research Scientist
glassdoor_job.JobTitle = glassdoor_job.JobTitle.replace('Statistician', 'Statistician/Research Scientist')
glassdoor_job.JobTitle = glassdoor_job.JobTitle.replace('Research Scientist', 'Statistician/Research Scientist')

glassdoor_job.JobTitle.value_counts()

Software Engineer                  16433
Business Analyst                    9610
Data Scientist                      6580
Data Engineer/DBA                   5855
Data Analyst                        5756
Statistician/Research Scientist     1099
Machine Learning Engineer            763
Name: JobTitle, dtype: int64

In [12]:
job_titles = ['Business Analyst', 'Data Analyst', 'Data Scientist', 
              'Data Engineer/DBA', 'Software Engineer', 'Statistician/Research Scientist']
cat_dtype = pd.api.types.CategoricalDtype(categories=job_titles)
glassdoor_job.JobTitle = glassdoor_job.JobTitle.astype(cat_dtype)

## 3. Job description
Extract skill information from description text

In [13]:
# Lowercase everything
glassdoor_job.JobDescription = glassdoor_job.JobDescription.str.lower()

In [14]:
# We need to find mentions to cloud platforms into Job Descriptions
# There are multiple ways to mention those, lets create a dictionary 
# to replace them and make standardized terms appear in Job Description
cloud_platforms = {
    'Alibaba': ' Alibaba Cloud ', 
    'Amazon Web Services': ' Amazon Web Services (AWS) ',
    'AWS': ' Amazon Web Services (AWS) ',
    'Google Cloud Platform': ' Google Cloud Platform (GCP) ', 
    'GCP': ' Google Cloud Platform (GCP) ',
    'Google Cloud': ' Google Cloud Platform (GCP) ',
    'IBM': ' IBM Cloud ', 
    'Azure': ' Microsoft Azure ', 
    'Oracle': ' Oracle Cloud ',
    'Red Hat': ' Red Hat Cloud ',
    'SAP': ' SAP Cloud ', 
    'Salesforce': ' Salesforce Cloud ', 
    'VMware': ' VMware Cloud '
}

# Replacing terms into Job Description
for find, repl in cloud_platforms.items():
    glassdoor_job.JobDescription = glassdoor_job.JobDescription.str.replace(find.lower(), repl.lower())

In [15]:
# Doing the same for databases
databases ={
    'dynamodb': ' aws dynamodb ',
    'dynamo': ' aws dynamodb ',
    ' rds ': ' aws relational database service ',
    'relational database service': ' aws relational database service ',
    'azure sql': ' azure sql database ',
    'google cloud sql': ' google cloud sql ',
    'microsoft access': ' microsoft access ', 
    'sql server': ' microsoft sql server ', 
    'my sql': ' mysql ', 
    'oracle db': ' oracle database ', 
    'postgres': ' postgressql ',
    'postgre': ' postgressql ',
    'postgre sql': ' postgressql ',
    'sqlite': 'sqlite '
}

for find, repl in databases.items():
    glassdoor_job.JobDescription = glassdoor_job.JobDescription.str.replace(find.lower(), repl.lower())

In [16]:
# Extract the skills we are interested in
skill_terms = ['Bash', ' C ', 'C\+\+', ' Java ', 'Javascript', 'MATLAB', 'Python', ' R ', 'SQL', 'TypeScript','Tableau', 'PowerBI', 'Excel', 
                   'Alibaba Cloud','Amazon Web Services (AWS)', 'Google Cloud Platform (GCP)', 'IBM Cloud', 'Microsoft Azure',
                   'Oracle Cloud', 'Red Hat Cloud', 'SAP Cloud', 'Salesforce Cloud', 'VMware Cloud',  
                   'aws dynamodb', 'aws relational database service', 'azure sql database', 'google cloud sql', 
                  'microsoft access', 'microsoft sql server', 'mysql', 'oracle database', 'postgressql', 'sqlite']

glassdoor_job.JobDescription = glassdoor_job.JobDescription.apply(str)

for i in skill_terms:
    mask = glassdoor_job.JobDescription.str.contains(i, flags=re.IGNORECASE,regex=True)
    glassdoor_job[i] = mask.to_frame()
    print("skill term "+ i + " is finished.")
    
glassdoor_job

skill term Bash is finished.
skill term  C  is finished.
skill term C\+\+ is finished.
skill term  Java  is finished.
skill term Javascript is finished.
skill term MATLAB is finished.
skill term Python is finished.
skill term  R  is finished.
skill term SQL is finished.
skill term TypeScript is finished.
skill term Tableau is finished.
skill term PowerBI is finished.
skill term Excel is finished.
skill term Alibaba Cloud is finished.


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


skill term Amazon Web Services (AWS) is finished.
skill term Google Cloud Platform (GCP) is finished.
skill term IBM Cloud is finished.
skill term Microsoft Azure is finished.
skill term Oracle Cloud is finished.
skill term Red Hat Cloud is finished.
skill term SAP Cloud is finished.
skill term Salesforce Cloud is finished.
skill term VMware Cloud is finished.
skill term aws dynamodb is finished.
skill term aws relational database service is finished.
skill term azure sql database is finished.
skill term google cloud sql is finished.
skill term microsoft access is finished.
skill term microsoft sql server is finished.
skill term mysql is finished.
skill term oracle database is finished.
skill term postgressql is finished.
skill term sqlite is finished.


,JobId,JobTitle,JobDescription,Bash,C,C\+\+,Java,Javascript,MATLAB,Python,...,aws dynamodb,aws relational database service,azure sql database,google cloud sql,microsoft access,microsoft sql server,mysql,oracle database,postgressql,sqlite
0,3.227100e+09,NaN,we are the drivers of the future who offer a g...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,3.406582e+09,NaN,<strong></strong>\n<br/><br/>\n<strong></stron...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3.230738e+09,Software Engineer,dottori.it &egrave; il motore di ricerca che t...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3.406677e+09,NaN,codup is looking for a senior sqa engineer for...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,3.201516e+09,NaN,<strong>department:</strong> research &amp; op...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165285,2.939045e+09,NaN,<strong>responsibilities</strong>\n\n<ul>\n<li...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
165286,3.301658e+09,NaN,<strong>job description</strong><br/><br/>make...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
165287,3.386398e+09,NaN,functie\n<br/>\nje ondersteunt de productiever...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
165288,3.402587e+09,NaN,het instituut voor engineering is &eacute;&eac...,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
# Adjust the 'C++' column name
glassdoor_job = glassdoor_job.rename(columns={'C\+\+':'C++'})
#Add cleaned job information into the main table
glassdoor_full[glassdoor_job.columns] = glassdoor_job

In [18]:
# Because the dataset is too large which makes the data loading process(to the aws server) quite slow(often returns the error of operation time out), 
# we decide to drop rows that are less usable to reduce the size.
glassdoor_new = glassdoor_full
# drop the rows where Jobtitle/country_code/sector/industry/jobid is na
# We only interested in the jobs related to 'Business Analyst', 'Data Analyst', 'Data Scientist', 'Data Engineer/DBA', 
# 'Software Engineer', 'Statistician/Research Scientist'; and with specific country
glassdoor_new1 = glassdoor_new[glassdoor_new['JobTitle'].notna()]
glassdoor_new2 = glassdoor_new1[glassdoor_new1['country_code'].notna()]
# Employer whose empSize is na does not complete its profile, so that there is no usable information about this kind of employer despite the employer's name
glassdoor_new3 = glassdoor_new2[glassdoor_new2['gaTrackerData.empSize'].notna()] 
glassdoor_new4 = glassdoor_new3[glassdoor_new3['overview.sector'].notna()]
glassdoor_new5 = glassdoor_new4[glassdoor_new4['gaTrackerData.industry'].notna()]
glassdoor_new6 = glassdoor_new5[glassdoor_new4['gaTrackerData.jobId.long'].notna()]
glassdoor = glassdoor_new6
glassdoor

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,benefits.benefitRatingDecimal,benefits.comments,benefits.highlights,benefits.numRatings,benefits.employerSummary,breadCrumbs,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,...,aws dynamodb,aws relational database service,azure sql database,google cloud sql,microsoft access,microsoft sql server,mysql,oracle database,postgressql,sqlite
13,2.000000,2814.0,2814.0,1,NaN,2814,-1,782762,Amaris,5001-10000,...,False,False,False,False,False,False,False,False,False,False
15,3.938710,2816.0,2816.0,311,NaN,2816,20081,4138,Accenture,10000--1,...,False,False,False,False,False,False,False,False,False,False
27,0.000000,2828.0,2828.0,0,NaN,2828,-1,790264,Derbysoft,51-200,...,False,False,False,False,False,False,False,False,False,False
29,0.000000,2830.0,2830.0,0,NaN,2830,10014,1437379,TrackTik,51-200,...,False,False,False,False,False,False,False,False,False,False
47,0.000000,2848.0,2848.0,0,NaN,2848,-1,648315,Nitor Infotech,201-500,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165236,4.140625,58347.0,58347.0,64,NaN,58347,-1,3035,Visa Inc.,10000--1,...,False,False,False,False,False,False,False,False,False,False
165260,0.000000,58371.0,58371.0,0,NaN,58371,-1,361256,Orange Logic*,51-200,...,False,False,False,False,False,False,False,False,False,False
165264,0.000000,58375.0,58375.0,0,NaN,58375,-1,651860,Eworks Recruitment Solutions,-1-0,...,False,False,False,False,False,False,False,False,False,False
165267,0.000000,58378.0,58378.0,0,NaN,58378,20007,442816,Advans International,1-50,...,False,False,False,False,False,False,False,False,False,False


# Create database tables

In [46]:
# Pass the connection string to a variable, conn_url
#list credentials here
hostname='cu-spring2020-group5.cggz75b61mlh.us-east-2.rds.amazonaws.com'
username='postgres'
password='postgres'
database='postgres'

# buildtheconnectionstring
conn_url='postgresql://'+username+':'+password+'@'+hostname+'/Glassdoor'

# Create an engine that connects to PostgreSQL server
engine = create_engine(conn_url)

# Establish a connection
connection = engine.connect()

# Pass the SQL statements that create all tables
stmt = """

DROP TABLE IF EXISTS job_type, skill_type, employer_size_type, employer_type,
  country,industry, sector, employer, employer_rating, employer_benefit_comments,
  employer_reviews, employer_competitors, job, job_salary, job_skill CASCADE;
  
CREATE TABLE job_type (
    type_id integer PRIMARY KEY,
    type varchar(50) NOT NULL
     );

CREATE TABLE skill_type (
    skill_id integer PRIMARY KEY,
    skill varchar(50) NOT NULL,
    category varchar(50)
     );

CREATE TABLE employer_size_type (
    type_id integer PRIMARY KEY,
    type varchar(50) NOT NULL
     );

CREATE TABLE employer_type (
    type_id integer PRIMARY KEY,
    type varchar(50) NOT NULL
     );

CREATE TABLE country (
    country_code char(3) PRIMARY KEY,
    country_name varchar(100) NOT NULL
     );

CREATE TABLE industry (
    industry_id integer PRIMARY KEY,
    industry varchar(100) NOT NULL
     );

CREATE TABLE sector (
    sector_id integer PRIMARY KEY,
    sector varchar(100) NOT NULL
     );

CREATE TABLE employer (
    emp_id integer PRIMARY KEY, 
    name varchar(100), 
    size_type_id integer REFERENCES employer_size_type(type_id),
    industry_id integer REFERENCES industry (industry_id),
    sector_id integer REFERENCES sector (sector_id),
    description text,
    mission text,
    founded_year numeric(4,0),
    emp_type_id integer REFERENCES employer_type(type_id)
 );

CREATE TABLE employer_rating (
    rating_id integer PRIMARY KEY, 
    emp_id integer REFERENCES employer (emp_id),
    rating_count integer,
    ceoapproval numeric(5,4),
    recommend numeric(5,4),
    star_rating numeric(3,2),
    benefit_rating_count integer,
    benefit_rating numeric(3,2),
    updated_time timestamp
);

CREATE TABLE employer_benefit_comments (
    comment_id integer PRIMARY KEY, 
    emp_id integer REFERENCES employer (emp_id),
    benefit_rating integer,
    comment text,
    created_time timestamp
);

CREATE TABLE employer_reviews (
    review_id integer PRIMARY KEY, 
    emp_id integer REFERENCES employer (emp_id),
    pros text,
    cons text,
    overall_rating integer,
    career_oppotunities integer,
    comp_benefits integer,
    culture_values integer,
    senior_management integer,
    worklife_balance integer,
    published_time date
);

CREATE TABLE employer_competitors(
    id integer PRIMARY KEY,
    emp_id integer REFERENCES employer (emp_id),
    competitor varchar(100)
);

CREATE TABLE job (
    job_id varchar(20) PRIMARY KEY, 
    job_title text,
    job_type_id integer REFERENCES job_type(type_id),
    country_code char(3) REFERENCES country(country_code),
    emp_id integer REFERENCES employer (emp_id)
);

CREATE TABLE job_salary (
    salary_id integer PRIMARY KEY, 
    job_id varchar(20) REFERENCES job(job_id),
    high integer,
    low integer,
    med integer,
    period varchar(10)
);

CREATE TABLE job_skill (
    job_id varchar(20) REFERENCES job(job_id),
    skill_id integer REFERENCES skill_type(skill_id),
    PRIMARY KEY(job_id,skill_id) 
);


"""

# Execute the statement to create tables and Close the connection
connection.execute(stmt)
connection.close()

# Extract, Transform and Load (ETL)

## 1. Category Tables
 To reduce the redundency and the size of of the employer table,  job table and job_skill table, we create these category tables. Category Tables include: job_type, skill_type, employer_size_type, employer_type, country,industry, sector. 


### · Table: job_type

In [20]:
# Job type defined in previous process
job_type = pd.DataFrame({'type_id' : [1,2,3,4,5,6], 'type' : ['Business Analyst', 'Data Analyst', 'Data Scientist', 
              'Data Engineer/DBA', 'Software Engineer', 'Statistician/Research Scientist']})
job_type

,type_id,type
0,1,Business Analyst
1,2,Data Analyst
2,3,Data Scientist
3,4,Data Engineer/DBA
4,5,Software Engineer
5,6,Statistician/Research Scientist


### · Table: skill_type
We add a new dimension to categorize the skills

In [21]:
# Job category
programming = ['Bash', ' C ', 'C++', ' Java ', 'Javascript', 'MATLAB', 'Python', ' R ', 'SQL', 'TypeScript']
visualization = ['Tableau', 'PowerBI', 'Excel']
cloud_computing = ['Alibaba Cloud','Amazon Web Services (AWS)', 'Google Cloud Platform (GCP)', 'IBM Cloud', 'Microsoft Azure',
                   'Oracle Cloud', 'Red Hat Cloud', 'SAP Cloud', 'Salesforce Cloud', 'VMware Cloud']
database =['aws dynamodb', 'aws relational database service', 'azure sql database', 'google cloud sql', 
                  'microsoft access', 'microsoft sql server', 'mysql', 'oracle database', 'postgressql', 'sqlite']
# Create dataframe for each skill category and combine them together
sk1 = pd.DataFrame({"skill": programming, "category": "programming language"})
sk2 = pd.DataFrame({"skill": visualization, "category": "visualization software"})
sk3 = pd.DataFrame({"skill": cloud_computing, "category": "cloud platform"})
sk4 = pd.DataFrame({"skill": database, "category": "database system"})
skill_type = pd.concat([sk1,sk2,sk3,sk4], axis=0)
# Add id for skill
skill_type.insert(0, 'skill_id', range(1, 1 + len(skill_type)))
skill_type.head()

,skill_id,skill,category
0,1,Bash,programming language
1,2,C,programming language
2,3,C++,programming language
3,4,Java,programming language
4,5,Javascript,programming language


### · Table:  employer_size_type

In [23]:
# Get the value of 'overview.size' and create the size type dataframe
employer_size_type = glassdoor["overview.size"].unique().tolist()
# ['201 to 500 employees', '5001 to 10000 employees', '10000+ employees', '51 to 200 employees',
# '1001 to 5000 employees', '501 to 1000 employees', '1 to 50 employees', 'Unknown']

# Create type_id for each size type and sort them.
employer_size_type = pd.DataFrame({"type_id": [0,1,2,3,4,5,6,7], 
                                   "type": ['Unknown','1 to 50 employees',  '51 to 200 employees','201 to 500 employees', 
                                    '501 to 1000 employees','1001 to 5000 employees','5001 to 10000 employees','10000+ employees']})
employer_size_type

,type_id,type
0,0,Unknown
1,1,1 to 50 employees
2,2,51 to 200 employees
3,3,201 to 500 employees
4,4,501 to 1000 employees
5,5,1001 to 5000 employees
6,6,5001 to 10000 employees
7,7,10000+ employees


### · Table: employer_type

In [24]:
# Get the value of 'overview.type' and create the employer type dataframe
employer_type =glassdoor["overview.type"].unique()
employer_type = np.sort(employer_type)
employer_type = pd.DataFrame({"type_id": list(range(1,len(employer_type)+1)), "type": list(employer_type)})
employer_type

,type_id,type
0,1,College / University
1,2,Company - Private
2,3,Company - Public
3,4,Contract
4,5,Franchise
5,6,Government
6,7,Hospital
7,8,Non-profit Organisation
8,9,Other Organisation
9,10,Private Practice / Firm


### · Table: country

In [24]:
# Create country dataframe
country = glassdoor[['country_code','country_name']].drop_duplicates().sort_values(by = 'country_code').reset_index(drop=True)
country.head()

,country_code,country_name
0,AFG,Afghanistan
1,ARE,United Arab Emirates
2,ARG,Argentina
3,AUS,Australia
4,AUT,Austria


### · Table: industry

In [25]:
# Create industry dataframe, use orginal industry id as the primary key
industry = glassdoor[['overview.industryId','overview.industry']].drop_duplicates().sort_values(by = "overview.industryId").reset_index(drop=True)
industry.columns = ['industry_id', 'industry']
industry

,industry_id,industry
0,200001,Accounting
1,200002,Legal
2,200003,Aerospace & Defence
3,200004,Animal Production
4,200006,Farm Support Services
...,...,...
116,200141,Passenger Rail
117,200144,Travel Agencies
118,200145,Grocery Shops & Supermarkets
119,200146,Venture Capital & Private Equity


### · Table: sector

In [26]:
# Create sector dataframe, use orginal sector id as the primary key
sector = glassdoor[['overview.sectorId','overview.sector']].drop_duplicates().sort_values(by = "overview.sectorId").reset_index(drop=True)
sector.columns = ['sector_id', 'sector']
sector

,sector_id,sector
0,10001,Accounting & Legal
1,10002,Aerospace & Defence
2,10003,Agriculture & Forestry
3,10004,"Arts, Entertainment & Recreation"
4,10005,Biotech & Pharmaceuticals
5,10006,Business Services
6,10007,"Building, Repair & Maintenance"
7,10008,Consumer Services
8,10009,Education
9,10010,Finance


## 2. Employer related tables
Employer relevant tables include: employer, employer_rating, employer_benefits, employer_competitors, employer_reviews
### · Table: employer

In [25]:
# Select the columns and drop duplicates rows; rename the columns
employer = glassdoor[['header.employerId','map.employerName','overview.size','overview.industryId',
                   'overview.sectorId','overview.description','overview.mission','overview.foundedYear','overview.type']].drop_duplicates()
employer.columns = ['emp_id', 'name', 'size_type','industry_id','sector_id','description','mission','founded_year','emp_type']

# Map size_type_id_list, add  size_type_id to employer dataframe and drop the size_type column
size_type_id_list = [employer_size_type.type_id[employer_size_type.type == i].values[0] for i in employer.size_type]
employer.insert(3, 'size_type_id', size_type_id_list)
employer = employer.drop('size_type', axis = 1)

# Map emp_type_id_list, add  emp_type_id to employer dataframe and drop the emp_type column
emp_type_id_list = [employer_type.type_id[employer_type.type == i].values[0] for i in employer.emp_type]
employer.insert(9, 'emp_type_id', emp_type_id_list)
employer = employer.drop('emp_type', axis = 1)

# Drop dupilcate rows and sort according to emp_id
employer = employer.drop_duplicates()
employer = employer.sort_values(by = 'emp_id').reset_index(drop=True)
employer

,emp_id,name,size_type_id,industry_id,sector_id,description,mission,founded_year,emp_type_id
0,12,Abbott Laboratories,7,200059,10012,Abbott is a global healthcare leader that help...,We help people live better and healthier with ...,1888,3
1,15,AMD,6,200060,10013,"At AMD, we are working to change the world for...","At AMD, our mission is simple: to become the r...",1969,3
2,35,American Express,7,200052,10010,"Each day, American Express makes it easier, sa...",American Express is the global services compan...,1850,3
3,40,AIG,7,200066,10014,NaN,NaN,1919,3
4,49,Analog Devices,7,200070,10015,"Analog Devices, Inc. (ADI) is a leading global...",NaN,1965,3
...,...,...,...,...,...,...,...,...,...
4351,3061400,Orderchamp,1,200033,10006,NaN,NaN,2019,2
4352,3077129,Audela (Canada),2,200061,10013,We are an AI company providing software soluti...,NaN,2016,2
4353,3080871,Rev-Trac,1,200061,10013,We help businesses deliver high quality SAP ap...,NaN,0,14
4354,3093450,Primis (Israel),2,200022,10006,"Primis is The Video Discovery Platform, helpin...",NaN,2007,2


### Table: employer_rating

In [21]:
# Select the columns; rename the columns
employer_rating = glassdoor[['header.employerId','rating.ceo.ratingsCount', 'rating.ceoApproval',
                                         'rating.recommendToFriend', 'rating.starRating', 'benefits.numRatings',
                                         'benefits.benefitRatingDecimal','job.discoverDate']]
employer_rating.columns = ['emp_id','rating_count','ceoapproval','recommend','star_rating',
                                        'benefit_rating_count','benefit_rating','updated_time']

# Clean the updated_time(convert 'Oct 20, 2019' into ''2019-10-20)                        
employer_rating['updated_time'] = [str(parse(i))[0:10] for i in employer_rating['updated_time']]
# Remove rows where the rating_count is 0(that employer does not have ratings)
employer_rating = employer_rating[-(employer_rating.rating_count == 0)]

# Drop duplicates and add the rating_id column
employer_rating = employer_rating.drop_duplicates().sort_values(by = ['emp_id','updated_time']).reset_index(drop=True)
employer_rating.insert(0, 'rating_id', range(1, 1 + len(employer_rating)))

# Convert the format of ratings
employer_rating.benefit_rating = round(employer_rating.benefit_rating,2)
employer_rating

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,rating_id,emp_id,rating_count,ceoapproval,recommend,star_rating,benefit_rating_count,benefit_rating,updated_time
0,1,12,811.0,0.82,0.66,3.5,7,4.86,2019-10-28
1,2,12,811.0,0.82,0.66,3.5,7,4.86,2019-10-31
2,3,12,811.0,0.82,0.66,3.5,7,4.86,2019-11-06
3,4,15,446.0,0.97,0.83,3.9,0,0.00,2019-10-25
4,5,15,446.0,0.97,0.83,3.9,0,0.00,2019-11-08
...,...,...,...,...,...,...,...,...,...
6404,6405,2919753,2.0,1.00,1.00,5.0,0,0.00,2019-10-25
6405,6406,2956748,6.0,1.00,1.00,5.0,0,0.00,2019-10-23
6406,6407,2981116,2.0,0.00,0.14,1.6,0,0.00,2019-10-14
6407,6408,3008542,1.0,1.00,1.00,5.0,0,0.00,2019-10-22


### · Table: employer_benefit_comments

In [29]:
# Read file 'glassdoor_benefits_comments.csv' 
benefit_comments = pd.read_csv('glassdoor_benefits_comments.csv') 

# Drop rows where index = nan. These rows do not have data except id column.
benefit_comments = benefit_comments[benefit_comments['index'].notna()]

# Extract the employer id and comment id from the main dataframe, and rename the columns
bcid_empid = glassdoor[['benefits.comments','header.employerId']]
bcid_empid.columns = ['id','emp_id']
# Inner join benefit_comments and bcid_empid on comment id
benefit_comments = pd.merge(bcid_empid,benefit_comments, on = 'id', how ='inner')

# Select the columns needed and drop duplicates
employer_benefit_comments = benefit_comments[['emp_id','index','benefits.comments.val.rating',
                                    'benefits.comments.val.comment','benefits.comments.val.createDate']]
employer_benefit_comments = employer_benefit_comments.drop_duplicates().sort_values(by=['emp_id','index']).reset_index(drop = True)

# Add a new comment_id for each comment, drop index column and rename other columns
employer_benefit_comments.insert(0,'comment_id',range(1,len(employer_benefit_comments)+1))
employer_benefit_comments = employer_benefit_comments.drop('index',axis = 1)
employer_benefit_comments.columns = ['comment_id', 'emp_id','benefit_rating','comment', 'created_time']
employer_benefit_comments

,comment_id,emp_id,benefit_rating,comment,created_time
0,1,12,5.0,Best is the eco-company car policy,2019-11-04 06:33:40.007
1,2,12,5.0,"Benefits package is excellent, the best I've h...",2019-03-16 09:46:24.663
2,3,12,5.0,"Holiday buy/sell option, pension contribution,...",2018-10-11 04:54:36.187
3,4,35,4.0,Great private health cover. Generally good bon...,2019-10-31 11:52:28.65
4,5,35,3.0,Just perks for a large company. Health insuran...,2019-10-06 12:44:58.593
...,...,...,...,...,...
2394,2395,2456544,3.0,"Standard benefits package, would be nice to ha...",2019-09-23 23:40:35.813
2395,2396,2482936,4.0,"Good Pension, holidays, yearly bonus, flexible...",2019-08-14 10:29:15.513
2396,2397,2549475,5.0,Great Business Park. Lovely teams who are incr...,2019-04-18 02:40:15.057
2397,2398,2582692,5.0,•Life assurance (pay out of 3x your salary) \n...,2019-09-15 13:12:30.843


### · Table: employer_reviews

In [51]:
# Read file 'glassdoor_reviews.csv'
reviews = pd.read_csv('glassdoor_reviews.csv') 
reviews = reviews[reviews['index'].notna()]#drop row where index = nan
# Drop rows where date is not date and convert into date type
reviews = reviews[-(reviews['reviews.val.date'].str.contains("week") 
                           | reviews['reviews.val.date'].str.contains("ago")
                           | reviews['reviews.val.date'].str.contains("day"))]
reviews['reviews.val.date'] = [parse(i) for i in reviews['reviews.val.date']]

# Add the employer id into the reviews table
review_empid = glassdoor[['reviews','header.employerId']]
review_empid.columns = ['id','emp_id']
reviews = pd.merge(review_empid,reviews, on = 'id', how ='inner')
#Select the columns needed
employer_reviews =reviews[['emp_id','index', 'reviews.val.pros', 'reviews.val.cons', 
                            'reviews.val.reviewRatings.overall', 
                           'reviews.val.reviewRatings.careerOpportunities',
                           'reviews.val.reviewRatings.compBenefits',
                           'reviews.val.reviewRatings.cultureValues',
                           'reviews.val.reviewRatings.seniorManagement',
                           'reviews.val.reviewRatings.worklifeBalance','reviews.val.date', ]]
employer_reviews = employer_reviews.drop_duplicates().sort_values(by=['emp_id','index']).reset_index(drop = True)
# Add a new review_id for each review, drop index, and rename columns
employer_reviews.insert(0,'review_id',range(1,len(employer_reviews)+1))
employer_reviews = employer_reviews.drop('index',axis = 1)
employer_reviews.columns = ['review_id', 'emp_id','pros','cons','overall_rating','career_oppotunities',
                            'comp_benefits','culture_values','senior_management','worklife_balance','published_time']
employer_reviews

,review_id,emp_id,pros,cons,overall_rating,career_oppotunities,comp_benefits,culture_values,senior_management,worklife_balance,published_time
0,1,12,great opportunities for career advancement and...,work life stress but things ease out after a w...,5.0,5.0,4.0,5.0,5.0,5.0,2019-04-14
1,2,12,"Goede verloning, voordelen en extra's\r\nUitge...",Weinig toekomstperspectief\r\nCommunicatie kan...,4.0,2.0,4.0,4.0,3.0,4.0,2018-10-27
2,3,12,Die Firma zahlt sehr gute Gehälter,Hierarchie sehr gross. Die Entscheidungen werd...,3.0,4.0,5.0,4.0,2.0,3.0,2017-07-25
3,4,12,Pleasant work environment with fluent interact...,Internal communication not always on point.,4.0,4.0,5.0,4.0,4.0,5.0,2018-12-13
4,5,12,"Friendly staff, lots of support from colleague...",work/life balance and very busy,4.0,4.0,5.0,5.0,2.0,2.0,2017-04-27
...,...,...,...,...,...,...,...,...,...,...,...
15940,15941,2981507,* Everyone has opportunity to build the produc...,* Look to get best hardware for employess.\r\n...,4.0,4.0,3.0,4.0,3.0,3.0,2019-08-27
15941,15942,2981507,1. Senior management is very supportive.\r\n2....,So far I have never seen any cons.,5.0,5.0,4.0,5.0,5.0,3.0,2019-09-04
15942,15943,2981507,1. Huge scope to learn and grow if someone is ...,I don't see any major cons. Everyone here is ...,5.0,5.0,5.0,5.0,5.0,5.0,2019-09-03
15943,15944,2981507,I have been working at SwoopTalent full-time....,"I don""t have any concern.",5.0,5.0,5.0,5.0,5.0,5.0,2019-08-28


### · Table: employer_competitors

In [31]:
# Read file 'glassdoor_reviews.csv'
competitors = pd.read_csv('glassdoor_overview_competitors.csv')
competitors = competitors[competitors['index'].notna()]#drop row where index = nan
# Add the employer id into the competitors table
competitor_empid = glassdoor[['overview.competitors','header.employerId']]
competitor_empid.columns = ['id','emp_id']
competitors = pd.merge(competitor_empid,competitors, on = 'id', how ='inner')
# Select the columns needed
employer_competitors =competitors[['emp_id','index','overview.competitors.val']]
employer_competitors = employer_competitors.drop_duplicates().sort_values(by=['emp_id']).reset_index(drop = True)
# Add competitor_id, drop index and rename the columns
employer_competitors.insert(0,'id',range(1,len(employer_competitors)+1))
employer_competitors = employer_competitors.drop('index',axis = 1)
employer_competitors.columns = ['id', 'emp_id','competitor']
employer_competitors

,id,emp_id,competitor
0,1,15,Qualcomm
1,2,15,Intel Corporation
2,3,15,NVIDIA
3,4,35,Mastercard
4,5,35,Visa Inc.
...,...,...,...
2430,2431,2549469,Zerodha
2431,2432,2566469,Zola Registry
2432,2433,2575465,Cisco Systems
2433,2434,2575465,Zoom Video Communications


## 3. Job related tables
Job relevant tables include: job, job_salary,job_skill
### · Table: job

In [32]:
# Select the columns, rename them and drop duplicates
job = glassdoor[['JobId','header.jobTitle','JobTitle','country_code','header.employerId','header.posted']]
job.columns = ['job_id', 'job_title', 'job_type','country_code','emp_id','posted_date']
job = job.drop_duplicates().sort_values(by = 'job_id').reset_index(drop =True)
# Convert the posted_date into date type
job['posted_date'] = [parse(i) for i in job['posted_date']]

# Map job_type_id_list; ass job_type_id to the job dataframe and drop the original job_type column
job_type_id_list = [job_type.type_id[job_type.type == i].values[0] for i in job.job_type]
job.insert(3, 'job_type_id', job_type_id_list)
job = job.drop('job_type', axis = 1)
job

,job_id,job_title,job_type_id,country_code,emp_id,posted_date
0,2.209223e+09,Software Engineer,5,BGR,334219,2019-10-28
1,2.209224e+09,Software Engineer,5,BRA,1161693,2019-10-28
2,2.209225e+09,Data Scientist,3,IRN,1002967,2019-10-28
3,2.209229e+09,Senior Software Engineer in Test,5,JOR,331673,2019-10-28
4,2.209230e+09,"Software Engineer, Senior",5,SGP,964424,2019-10-28
...,...,...,...,...,...,...
12283,3.412762e+09,Business Analyst,1,AUS,573105,2019-11-10
12284,3.412764e+09,RWA Statistician,6,FRA,254568,2019-11-10
12285,3.412764e+09,Software Engineer,5,ROU,877317,2019-11-10
12286,3.412766e+09,Product Data Analyst,2,IND,1121475,2019-11-10


### · Table: job_salary

In [33]:
# Select the columns, rename them and drop duplicates
salary = glassdoor[['JobId','header.payHigh','header.payLow','header.payMed','header.payPeriod']]
salary.columns = ['job_id','high','low','med','period']
job_salary = salary[salary.high.notna()].drop_duplicates().sort_values(by = 'job_id').reset_index(drop=True)
job_salary.insert(0, 'salary_id', range(1, 1 + len(job_salary)))
job_salary

,salary_id,job_id,high,low,med,period
0,1,2.285345e+09,229122.0,189140.0,210191.0,ANNUAL
1,2,2.396578e+09,136756.0,81121.0,101016.0,ANNUAL
2,3,2.756013e+09,141396.0,102166.0,121709.0,ANNUAL
3,4,2.798355e+09,146509.0,86841.0,108316.0,ANNUAL
4,5,2.823230e+09,109620.0,74792.0,90679.0,ANNUAL
...,...,...,...,...,...,...
344,345,3.412234e+09,73732.0,45091.0,58318.0,ANNUAL
345,346,3.412351e+09,93289.0,58410.0,73033.0,ANNUAL
346,347,3.412389e+09,99000.0,88000.0,93500.0,NaN
347,348,3.412394e+09,140071.0,97016.0,117404.0,ANNUAL


### · Table: job_skill

In [35]:
job_skill = glassdoor[glassdoor_job.columns].drop(['JobTitle','JobDescription'], axis = 1)
# Reshape the job_skill into the format{job_id,skill,value(True/False)}
job_skill = job_skill.melt(id_vars = 'JobId')
job_skill = job_skill[job_skill.value == True].drop_duplicates().drop('value',axis = 1).reset_index(drop = True)

# Map skill_id_list; add job_type_id to the job_skill dataframe
skill_id_list = [skill_type.skill_id[skill_type.skill == i].values[0] for i in job_skill.variable]
job_skill.insert(1, 'skill_id', skill_id_list)
job_skill = job_skill.drop('variable', axis = 1)
job_skill.columns = ['job_id','skill_id']
job_skill

,job_id,skill_id
0,3.390923e+09,1
1,3.269267e+09,1
2,3.340748e+09,1
3,3.390141e+09,1
4,3.304180e+09,1
...,...,...
26431,3.349427e+09,33
26432,3.290652e+09,33
26433,3.200549e+09,33
26434,3.357208e+09,33


# Insert data

In [47]:
job_type.to_sql(name='job_type', con=engine, if_exists='append', index=False)
skill_type.to_sql(name='skill_type', con=engine, if_exists='append', index=False)
employer_size_type.to_sql(name='employer_size_type', con=engine, if_exists='append', index=False)
employer_type.to_sql(name='employer_type', con=engine, if_exists='append', index=False)
country.to_sql(name='country', con=engine, if_exists='append', index=False)
industry.to_sql(name='industry', con=engine, if_exists='append', index=False)
sector.to_sql(name='sector', con=engine, if_exists='append', index=False)

In [48]:
employer.to_sql(name='employer', con=engine, if_exists='append', index=False)
employer_rating.to_sql(name='employer_rating', con=engine, if_exists='append', index=False)
employer_benefit_comments.to_sql(name='employer_benefit_comments', con=engine, if_exists='append', index=False)
employer_reviews.to_sql(name='employer_reviews', con=engine, if_exists='append', index=False)
employer_competitors.to_sql(name='employer_competitors', con=engine, if_exists='append', index=False)

In [49]:
job.to_sql(name='job', con=engine, if_exists='append', index=False)
job_salary.to_sql(name='job_salary', con=engine, if_exists='append', index=False)
job_skill.to_sql(name='job_skill', con=engine, if_exists='append', index=False)